In [18]:
import pandas as pd
import os


pd.read_csv(os.path.dirname(os.getcwd())+"\\outputs\\stock_prices_historical.csv").groupby(["load_datetime"]).count()


,Unnamed: 0,_symbol,_name,_current_price,_previous_close,_52_week_high,_52_week_low,_pe,_2023_cash_div
load_datetime,,,,,,,,,
2024-12-19 22:23:44,360,360,294,360,360,360,360,183,125
2024-12-19 22:35:45,360,360,294,360,360,360,360,183,125
2024-12-19 22:47:35,360,360,294,360,360,360,360,183,125
load_datetime,0,2,2,2,2,2,2,2,2


In [23]:
import pandas as pd
import os

# read raw
raw=pd.read_csv(os.path.dirname(os.getcwd())+"\\outputs\\stock_prices_historical.csv", header=None)
raw.columns=headers_f
raw.head()


,ix,_symbol,_name,_current_price,_previous_close,_52_week_high,_52_week_low,_pe,_2023_cash_div,load_datetime
0,0,8990B,NaN,100.90,100.90,101.00,90.10,NaN,NaN,2024-12-19 23:04:15
1,1,8990P,8990 Holdings - Series A Perpetual Pref,101.10,101.10,101.90,95.05,NaN,NaN,2024-12-19 23:04:15
2,2,AAA,Asia Amalgamated Holdings Corp.,1.61,1.61,2.25,1.21,-35.698,NaN,2024-12-19 23:04:15
3,3,AB,"Atok-Big Wedge Company, Inc.",4.71,4.75,6.00,2.51,747.619,NaN,2024-12-19 23:04:15
4,4,ABA,"AbaCore Capital Holdings, Inc.",0.53,0.55,1.40,0.50,2.386,NaN,2024-12-19 23:04:15


In [25]:
# format data
raw["load_date"]=raw["load_datetime"].apply(lambda x:x.split(" ")[0])
numcols=['_current_price', '_previous_close', '_52_week_high', '_52_week_low']
for col in numcols:
    raw[col]=raw[col].astype(float)

# get average, min, max per day
cleaned=raw.groupby(["load_date", "_symbol", "_name"]).agg(
    current_day_price_avg=('_current_price', 'mean')
    , current_day_price_min=('_current_price', 'min')
    , current_day_price_max=('_current_price', 'max')    
    , previous_day_close=('_previous_close', 'mean')
    , _52_wk_high_avg=('_52_week_high', 'mean')
    , _52_wk_low_avg=('_52_week_low', 'mean')
    , rec_count=('_symbol', 'count')
)
print(cleaned.head())
# apply(lambda x:float(x.split(" ")[0]))

                                                            current_day_price_avg  \
load_date  _symbol _name                                                            
2024-12-19 8990P   8990 Holdings - Series A Perpetual Pref                 101.10   
           AAA     Asia Amalgamated Holdings Corp.                           1.61   
           AB      Atok-Big Wedge Company, Inc.                              4.71   
           ABA     AbaCore Capital Holdings, Inc.                            0.53   
           ABG     Asiabest Group International, Inc.                       26.20   

                                                            current_day_price_min  \
load_date  _symbol _name                                                            
2024-12-19 8990P   8990 Holdings - Series A Perpetual Pref                 101.10   
           AAA     Asia Amalgamated Holdings Corp.                           1.61   
           AB      Atok-Big Wedge Company, Inc.                 

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd


# Initialize WebDriver
options = Options()
options.headless = True  # Optional: Run headless (without UI)
driver = webdriver.Chrome(options=options)

# Open the page
driver.get('https://www.pesobility.com/stock')
time.sleep(10)

# Get headers
heads=driver.find_elements(By.XPATH, '//thead//th')
headers=[]
for i in range(0, len(heads)):
    headers.append(heads[i].text)
headers_f=["_"+x.replace(" ", "_").replace("(%)", "").replace("-", "_").lower().strip("_") for x in headers]
headers_f=["ix"]+headers_f+["load_datetime"]
headers_f

['ix',
 '_symbol',
 '_name',
 '_current_price',
 '_previous_close',
 '_52_week_high',
 '_52_week_low',
 '_pe',
 '_2023_cash_div',
 'load_datetime']